# THIS FILE IS FOR MERGING EXT DATA

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Counter train and test sets

In [2]:
data_train = pd.read_parquet(Path("data") / "train.parquet")
data_test = pd.read_parquet(Path("data") / "test.parquet")

In [3]:
data_all = pd.concat([data_train, data_test])
data_all = data_all.sort_values(by=['counter_name', 'date'])

In [4]:
data_all.date.max()

Timestamp('2021-09-09 23:00:00')

In [5]:
data_train.date.max()

Timestamp('2021-08-09 23:00:00')

In [6]:
data_test.date.max()

Timestamp('2021-09-09 23:00:00')

In [7]:
data_all.nunique(axis=0)

counter_id                     56
counter_name                   56
site_id                        30
site_name                      30
bike_count                    998
date                         8973
counter_installation_date      22
coordinates                    30
counter_technical_id           30
latitude                       30
longitude                      30
log_bike_count                998
dtype: int64

# Creating WW_mix

In [8]:
weather = pd.read_csv('ext_data/weather.csv')
weather["date"] = pd.to_datetime(weather["date"], format = "%Y-%m-%d %H:%M:%S")
weather = weather.loc[:, weather.columns.intersection(['date','ww','w1','w2'])]
weather

,date,ww,w1,w2
0,2021-01-01 00:00:00,2,0.0,0.0
1,2021-01-01 03:00:00,40,0.0,0.0
2,2021-01-01 06:00:00,3,1.0,1.0
3,2021-01-01 09:00:00,10,1.0,1.0
4,2021-01-01 12:00:00,2,2.0,2.0
...,...,...,...,...
3317,2020-09-30 09:00:00,3,2.0,2.0
3318,2020-09-30 12:00:00,1,2.0,2.0
3319,2020-09-30 15:00:00,1,2.0,2.0
3320,2020-09-30 18:00:00,3,2.0,2.0


In [9]:
weather.nunique(axis=0)

date    3321
ww        42
w1         9
w2         9
dtype: int64

In [10]:
weather.date.max()

Timestamp('2021-10-21 12:00:00')

In [11]:
weather[weather.duplicated(keep=False)]

,date,ww,w1,w2
2017,2020-11-20 18:00:00,1,1.0,1.0
2018,2020-11-20 18:00:00,1,1.0,1.0


In [12]:
weather.drop(2017, inplace=True)

In [13]:
weather.isna().sum()

date     0
ww       0
w1       7
w2      10
dtype: int64

In [14]:
weather.dropna(inplace = True)

In [15]:
weather_0 = weather.loc[:, weather.columns.intersection(['date','ww'])]
weather_1 = weather.loc[:, weather.columns.intersection(['date','w1'])]
weather_2 = weather.loc[:, weather.columns.intersection(['date','w2'])]

In [16]:
weather_1["date"] = weather_1["date"] + pd.Timedelta(hours=-1)
weather_2["date"] = weather_2["date"] + pd.Timedelta(hours=-2)

In [17]:
weather_0.nunique(axis=0)

date    3311
ww        42
dtype: int64

 # Conf Daily 1*0

In [18]:
confinement = pd.read_csv('ext_data\confinement.csv', sep=';')

In [19]:
confinement["date"] = pd.to_datetime(confinement["date"], format = "%Y-%m-%d %H:%M:%S")

In [20]:
confinement.drop('Unnamed: 0', axis=1,inplace=True)
confinement

,date,Confinement ce jour
0,2020-09-01 00:00:00,0
1,2020-09-01 01:00:00,0
2,2020-09-01 02:00:00,0
3,2020-09-01 03:00:00,0
4,2020-09-01 04:00:00,0
...,...,...
9884,2021-10-17 20:00:00,0
9885,2021-10-17 21:00:00,0
9886,2021-10-17 22:00:00,0
9887,2021-10-17 23:00:00,0


In [21]:
confinement.rename({'Confinement ce jour':'conf'}, axis=1, inplace = True)

# Merge ww_mix and conf daily

In [22]:
conf_ww = pd.merge(confinement, weather_2, how='left', on='date')
conf_ww = pd.merge(conf_ww, weather_1, how='left', on='date')
conf_ww = pd.merge(conf_ww, weather_0, how='left', on='date')
conf_ww.replace(np.nan, 0, inplace=True)
conf_ww["ww_mix"] = conf_ww["w1"] + conf_ww["w2"] + conf_ww["ww"] 

In [23]:
conf_ww

,date,conf,w2,w1,ww,ww_mix
0,2020-09-01 00:00:00,0,0.0,0.0,1.0,1.0
1,2020-09-01 01:00:00,0,0.0,0.0,0.0,0.0
2,2020-09-01 02:00:00,0,0.0,0.0,0.0,0.0
3,2020-09-01 03:00:00,0,0.0,0.0,2.0,2.0
4,2020-09-01 04:00:00,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
9884,2021-10-17 20:00:00,0,0.0,0.0,0.0,0.0
9885,2021-10-17 21:00:00,0,0.0,0.0,1.0,1.0
9886,2021-10-17 22:00:00,0,0.0,0.0,0.0,0.0
9887,2021-10-17 23:00:00,0,0.0,0.0,0.0,0.0


In [24]:
conf_ww.drop(columns=['ww','w2','w1'], inplace=True)

In [25]:
conf_ww

,date,conf,ww_mix
0,2020-09-01 00:00:00,0,1.0
1,2020-09-01 01:00:00,0,0.0
2,2020-09-01 02:00:00,0,0.0
3,2020-09-01 03:00:00,0,2.0
4,2020-09-01 04:00:00,0,0.0
...,...,...,...
9884,2021-10-17 20:00:00,0,0.0
9885,2021-10-17 21:00:00,0,1.0
9886,2021-10-17 22:00:00,0,0.0
9887,2021-10-17 23:00:00,0,0.0


In [26]:
conf_ww.to_csv(r'C:\Users\CEDI\Dropbox\Education\X\Python for Data Science\bike_counters\ext_data\ext_2f.csv')

# Adding weather

In [27]:
w12 = pd.read_csv('ext_data/weather.csv')
w12["date"] = pd.to_datetime(w12["date"], format = "%Y-%m-%d %H:%M:%S")
w12 = w12.loc[:, w12.columns.intersection(['date', 'pmer', 'ff', 't', 'u','vv', 'ww',
                                                       'nbas', 'pres','raf10','etat_sol','rr1','rr3'])]
w12

,date,pmer,ff,t,u,vv,ww,nbas,pres,raf10,etat_sol,rr1,rr3
0,2021-01-01 00:00:00,100810,1.8,272.75,96,990,2,1.0,99680,2.5,1.0,0.0,0.0
1,2021-01-01 03:00:00,100920,1.7,271.25,98,210,40,1.0,99790,2.2,1.0,0.0,0.0
2,2021-01-01 06:00:00,100950,2.6,271.95,98,3660,3,5.0,99820,3.2,1.0,0.0,0.0
3,2021-01-01 09:00:00,101100,1.7,272.45,97,3500,10,1.0,99970,2.3,13.0,0.0,0.2
4,2021-01-01 12:00:00,101110,1.0,276.95,82,8000,2,7.0,100000,2.5,11.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,2020-09-30 09:00:00,101540,4.4,289.95,82,18000,3,7.0,100480,7.1,0.0,0.0,0.0
3318,2020-09-30 12:00:00,101320,4.9,292.05,66,25000,1,7.0,100270,7.2,0.0,0.0,0.2
3319,2020-09-30 15:00:00,101140,4.1,291.55,72,25000,1,7.0,100090,6.4,0.0,0.0,0.0
3320,2020-09-30 18:00:00,101020,2.7,290.15,73,40820,3,8.0,99960,4.3,0.0,0.0,0.0


# Adding hourly

In [28]:
hourly = pd.read_csv('ext_data/external_data_hourly.csv')
hourly["date"] = pd.to_datetime(hourly["date"], format = "%m/%d/%Y %H:%M")

In [29]:
hourly

,date,hourly
0,2020-09-01 00:00:00,0
1,2020-09-01 01:00:00,0
2,2020-09-01 02:00:00,0
3,2020-09-01 03:00:00,0
4,2020-09-01 04:00:00,0
...,...,...
9884,2021-10-17 20:00:00,0
9885,2021-10-17 21:00:00,0
9886,2021-10-17 22:00:00,0
9887,2021-10-17 23:00:00,0


# Adding intensity

In [30]:
intense = pd.read_csv('ext_data/conf_intensity_g2.csv', sep = ',')
intense["date"] = pd.to_datetime(intense["date"], format = "%m/%d/%Y %H:%M")
intense.drop(columns=['Unnamed: 0','Phase'], axis=1,inplace=True)
#w12 = w12.loc[:, w12.columns.intersection(['date', 'pmer', 'ff', 't', 'u','vv', 'ww',
#                                                       'nbas', 'pres','raf10','etat_sol','rr1','rr3'])]
intense

,date,Intensev1,Intensev2
0,2020-09-01 00:00:00,0,0
1,2020-09-01 01:00:00,0,0
2,2020-09-01 02:00:00,0,0
3,2020-09-01 03:00:00,0,0
4,2020-09-01 04:00:00,0,0
...,...,...,...
9884,2021-10-17 20:00:00,0,0
9885,2021-10-17 21:00:00,0,0
9886,2021-10-17 22:00:00,0,0
9887,2021-10-17 23:00:00,0,0


In [31]:
ext_conf12 = pd.merge_asof(
        conf_ww.sort_values("date"), w12.sort_values("date"), on="date"
)

In [32]:
ext_conf12 = pd.merge_asof(
        ext_conf12, hourly.sort_values("date"), on="date"
)

In [33]:
ext_intense12 = pd.merge_asof(
        ext_conf12, intense, on="date"
)

In [34]:
ext_intense12

,date,conf,ww_mix,pmer,ff,t,u,vv,ww,nbas,pres,raf10,etat_sol,rr1,rr3,hourly,Intensev1,Intensev2
0,2020-09-01 00:00:00,0,1.0,102050,1.6,285.75,81,30000,1,0.0,100960,2.4,0.0,0.0,0.0,0,0,0
1,2020-09-01 01:00:00,0,0.0,102050,1.6,285.75,81,30000,1,0.0,100960,2.4,0.0,0.0,0.0,0,0,0
2,2020-09-01 02:00:00,0,0.0,102050,1.6,285.75,81,30000,1,0.0,100960,2.4,0.0,0.0,0.0,0,0,0
3,2020-09-01 03:00:00,0,2.0,101990,1.1,283.95,88,25000,2,0.0,100900,1.5,0.0,0.0,0.0,0,0,0
4,2020-09-01 04:00:00,0,0.0,101990,1.1,283.95,88,25000,2,0.0,100900,1.5,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9884,2021-10-17 20:00:00,0,0.0,102050,2.2,285.65,68,25000,3,0.0,100960,3.0,0.0,0.0,0.0,0,0,0
9885,2021-10-17 21:00:00,0,1.0,102120,1.1,282.75,82,25000,1,0.0,101020,2.6,0.0,0.0,0.0,0,0,0
9886,2021-10-17 22:00:00,0,0.0,102120,1.1,282.75,82,25000,1,0.0,101020,2.6,0.0,0.0,0.0,0,0,0
9887,2021-10-17 23:00:00,0,0.0,102120,1.1,282.75,82,25000,1,0.0,101020,2.6,0.0,0.0,0.0,0,0,0


In [35]:
ext_intense12['t'] -= 273.15

In [36]:
ext_intense12.isna().sum()

date           0
conf           0
ww_mix         0
pmer           0
ff             0
t              0
u              0
vv             0
ww             0
nbas          15
pres           0
raf10         30
etat_sol     156
rr1           27
rr3           15
hourly         0
Intensev1      0
Intensev2      0
dtype: int64

# We replace all the NaNs by 0's.

In [37]:
ext_intense12.replace(np.nan, 0, inplace=True)

In [38]:
#x = np.arange(1,9890)
#y = ext_conf12['raf10']
#plt.plot(x,y)

In [39]:
ext_intense12

,date,conf,ww_mix,pmer,ff,t,u,vv,ww,nbas,pres,raf10,etat_sol,rr1,rr3,hourly,Intensev1,Intensev2
0,2020-09-01 00:00:00,0,1.0,102050,1.6,12.6,81,30000,1,0.0,100960,2.4,0.0,0.0,0.0,0,0,0
1,2020-09-01 01:00:00,0,0.0,102050,1.6,12.6,81,30000,1,0.0,100960,2.4,0.0,0.0,0.0,0,0,0
2,2020-09-01 02:00:00,0,0.0,102050,1.6,12.6,81,30000,1,0.0,100960,2.4,0.0,0.0,0.0,0,0,0
3,2020-09-01 03:00:00,0,2.0,101990,1.1,10.8,88,25000,2,0.0,100900,1.5,0.0,0.0,0.0,0,0,0
4,2020-09-01 04:00:00,0,0.0,101990,1.1,10.8,88,25000,2,0.0,100900,1.5,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9884,2021-10-17 20:00:00,0,0.0,102050,2.2,12.5,68,25000,3,0.0,100960,3.0,0.0,0.0,0.0,0,0,0
9885,2021-10-17 21:00:00,0,1.0,102120,1.1,9.6,82,25000,1,0.0,101020,2.6,0.0,0.0,0.0,0,0,0
9886,2021-10-17 22:00:00,0,0.0,102120,1.1,9.6,82,25000,1,0.0,101020,2.6,0.0,0.0,0.0,0,0,0
9887,2021-10-17 23:00:00,0,0.0,102120,1.1,9.6,82,25000,1,0.0,101020,2.6,0.0,0.0,0.0,0,0,0


# Adding Holidays, Bank Holidays and Strike rate

In [40]:
bvs = pd.read_csv('ext_data/bank_vacs_strike.csv')
bvs["date"] = pd.to_datetime(bvs["date"], format = "%m/%d/%Y %H:%M")
bvs

,date,vacances_zone_c,bank_days,strike_rate
0,2020-09-01 00:00:00,0,0,0.0
1,2020-09-01 01:00:00,0,0,0.0
2,2020-09-01 02:00:00,0,0,0.0
3,2020-09-01 03:00:00,0,0,0.0
4,2020-09-01 04:00:00,0,0,0.0
...,...,...,...,...
9882,2021-10-17 20:00:00,0,0,NaN
9883,2021-10-17 21:00:00,0,0,NaN
9884,2021-10-17 22:00:00,0,0,NaN
9885,2021-10-17 23:00:00,0,0,NaN


In [41]:
ext_intense12 = pd.merge_asof(
        ext_intense12, bvs, on="date"
)

In [42]:
ext_intense12

,date,conf,ww_mix,pmer,ff,t,u,vv,ww,nbas,...,raf10,etat_sol,rr1,rr3,hourly,Intensev1,Intensev2,vacances_zone_c,bank_days,strike_rate
0,2020-09-01 00:00:00,0,1.0,102050,1.6,12.6,81,30000,1,0.0,...,2.4,0.0,0.0,0.0,0,0,0,0,0,0.0
1,2020-09-01 01:00:00,0,0.0,102050,1.6,12.6,81,30000,1,0.0,...,2.4,0.0,0.0,0.0,0,0,0,0,0,0.0
2,2020-09-01 02:00:00,0,0.0,102050,1.6,12.6,81,30000,1,0.0,...,2.4,0.0,0.0,0.0,0,0,0,0,0,0.0
3,2020-09-01 03:00:00,0,2.0,101990,1.1,10.8,88,25000,2,0.0,...,1.5,0.0,0.0,0.0,0,0,0,0,0,0.0
4,2020-09-01 04:00:00,0,0.0,101990,1.1,10.8,88,25000,2,0.0,...,1.5,0.0,0.0,0.0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9884,2021-10-17 20:00:00,0,0.0,102050,2.2,12.5,68,25000,3,0.0,...,3.0,0.0,0.0,0.0,0,0,0,0,0,NaN
9885,2021-10-17 21:00:00,0,1.0,102120,1.1,9.6,82,25000,1,0.0,...,2.6,0.0,0.0,0.0,0,0,0,0,0,NaN
9886,2021-10-17 22:00:00,0,0.0,102120,1.1,9.6,82,25000,1,0.0,...,2.6,0.0,0.0,0.0,0,0,0,0,0,NaN
9887,2021-10-17 23:00:00,0,0.0,102120,1.1,9.6,82,25000,1,0.0,...,2.6,0.0,0.0,0.0,0,0,0,0,0,NaN


In [43]:
ext_intense12.isna().sum()

date                 0
conf                 0
ww_mix               0
pmer                 0
ff                   0
t                    0
u                    0
vv                   0
ww                   0
nbas                 0
pres                 0
raf10                0
etat_sol             0
rr1                  0
rr3                  0
hourly               0
Intensev1            0
Intensev2            0
vacances_zone_c      0
bank_days            0
strike_rate        913
dtype: int64

In [44]:
ext_intense12.strike_rate.replace(np.nan, 0, inplace=True)

In [45]:
ext_intense12

,date,conf,ww_mix,pmer,ff,t,u,vv,ww,nbas,...,raf10,etat_sol,rr1,rr3,hourly,Intensev1,Intensev2,vacances_zone_c,bank_days,strike_rate
0,2020-09-01 00:00:00,0,1.0,102050,1.6,12.6,81,30000,1,0.0,...,2.4,0.0,0.0,0.0,0,0,0,0,0,0.0
1,2020-09-01 01:00:00,0,0.0,102050,1.6,12.6,81,30000,1,0.0,...,2.4,0.0,0.0,0.0,0,0,0,0,0,0.0
2,2020-09-01 02:00:00,0,0.0,102050,1.6,12.6,81,30000,1,0.0,...,2.4,0.0,0.0,0.0,0,0,0,0,0,0.0
3,2020-09-01 03:00:00,0,2.0,101990,1.1,10.8,88,25000,2,0.0,...,1.5,0.0,0.0,0.0,0,0,0,0,0,0.0
4,2020-09-01 04:00:00,0,0.0,101990,1.1,10.8,88,25000,2,0.0,...,1.5,0.0,0.0,0.0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9884,2021-10-17 20:00:00,0,0.0,102050,2.2,12.5,68,25000,3,0.0,...,3.0,0.0,0.0,0.0,0,0,0,0,0,0.0
9885,2021-10-17 21:00:00,0,1.0,102120,1.1,9.6,82,25000,1,0.0,...,2.6,0.0,0.0,0.0,0,0,0,0,0,0.0
9886,2021-10-17 22:00:00,0,0.0,102120,1.1,9.6,82,25000,1,0.0,...,2.6,0.0,0.0,0.0,0,0,0,0,0,0.0
9887,2021-10-17 23:00:00,0,0.0,102120,1.1,9.6,82,25000,1,0.0,...,2.6,0.0,0.0,0.0,0,0,0,0,0,0.0


In [46]:
ext_intense20 = ext_intense12

# On ajoute TRAFIC

In [47]:
trafic = pd.read_csv('ext_data/trafic_journalier.csv')
trafic["date"] = pd.to_datetime(trafic["date"], format = "%m/%d/%Y")
trafic

,date,Vehicules
0,2020-09-01,81.338753
1,2020-09-02,84.253298
2,2020-09-03,87.538071
3,2020-09-04,85.891247
4,2020-09-05,81.749296
...,...,...
407,2021-10-14,80.546679
408,2021-10-15,75.336157
409,2021-10-16,74.553766
410,2021-10-17,66.418217


In [48]:
len(trafic)

412

In [49]:
for i in range(len(trafic)):
    if (trafic.loc[i, "date"] > pd.to_datetime("2020/10/21")) & (trafic.loc[i, "date"] < pd.to_datetime("2021/09/01")):
        trafic.loc[i,'Vehicules'] = 0


In [50]:
trafic

,date,Vehicules
0,2020-09-01,81.338753
1,2020-09-02,84.253298
2,2020-09-03,87.538071
3,2020-09-04,85.891247
4,2020-09-05,81.749296
...,...,...
407,2021-10-14,80.546679
408,2021-10-15,75.336157
409,2021-10-16,74.553766
410,2021-10-17,66.418217


In [51]:
ext_intense20 = pd.merge_asof(
        ext_intense20, trafic, on="date"
)

In [52]:
ext_intense20.to_csv(r'C:\Users\CEDI\Dropbox\Education\X\Python for Data Science\bike_counters\ext_data\ext_intense20.csv')